## 5.6 GPU

先看如何使用单个GPU进行计算

In [1]:
!nvidia-smi

Tue Apr 16 13:15:04 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.67                 Driver Version: 550.67         CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 3090        On  |   00000000:01:00.0 Off |                  N/A |
| 30%   21C    P8             17W /  350W |       1MiB /  24576MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

### 5.6.1 计算设备

可以指定用于存储和计算的设备，默认情况下张量是在内存中创建的，并使用CPU进行计算。

在PyTorch中，可以使用`torch.device('cpu')`和`torch.device('cuda')`。前者代表所有CPU，而后者只代表一个GPU。可以使用`torch.device(f'cuda:{i}')`代表第i块GPU。

In [2]:
import torch
from torch import nn

torch.device('cpu'), torch.device('cuda'), torch.device('cuda:1')

(device(type='cpu'), device(type='cuda'), device(type='cuda', index=1))

也可以查询可用GPU的数量

In [3]:
torch.cuda.device_count()

6

现在我们定义了两个方便的函数， 这两个函数允许我们在不存在所需所有GPU的情况下运行代码。

In [4]:
def try_gpu(i=0):  #@save
    """如果存在，则返回gpu(i)，否则返回cpu()"""
    if torch.cuda.device_count() >= i + 1:
        return torch.device(f'cuda:{i}')
    return torch.device('cpu')

def try_all_gpus():  #@save
    """返回所有可用的GPU，如果没有GPU，则返回[cpu(),]"""
    devices = [torch.device(f'cuda:{i}')
             for i in range(torch.cuda.device_count())]
    return devices if devices else [torch.device('cpu')]

try_gpu(), try_gpu(10), try_all_gpus()

(device(type='cuda', index=0),
 device(type='cpu'),
 [device(type='cuda', index=0),
  device(type='cuda', index=1),
  device(type='cuda', index=2),
  device(type='cuda', index=3),
  device(type='cuda', index=4),
  device(type='cuda', index=5)])

### 5.6.2 张量与GPU

可以查询张量所在的设备，默认是在CPU

In [5]:
x = torch.tensor([1, 2, 3])
x.device

device(type='cpu')

#### 5.6.2.1 存储在GPU上

下面在第一个GPU上创建张量x

In [6]:
X = torch.ones(2, 3, device=try_gpu())
X

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')

下面在第二个GPU上创建一个随机张量

In [7]:
Y = torch.rand(2, 3, device=try_gpu(1))
Y

tensor([[0.0174, 0.5500, 0.3363],
        [0.7197, 0.7289, 0.8981]], device='cuda:1')

#### 5.6.2.2 复制

只能在同一个设备上进行运算，否则要复制挪动

In [8]:
Z = X.cuda(1)
print(X)
print(Z)

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')
tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:1')


In [11]:
Y + Z

tensor([[1.0174, 1.5500, 1.3363],
        [1.7197, 1.7289, 1.8981]], device='cuda:1')

如果复制到原来的那块GPU，则不会分配新内存

In [12]:
Z.cuda(1) is Z

True

#### 5.6.2.3 旁注

数据在不同设备的传输是缓慢的，而且NumPy会将所有设备复制到CPU中再进行操作

### 5.6.3 神经网络与GPU

神经网络也可以放

In [13]:
net = nn.Sequential(
    nn.Linear(3, 1)
)

net = net.to(device=try_gpu())

In [14]:
net(X)

tensor([[-0.0989],
        [-0.0989]], device='cuda:0', grad_fn=<AddmmBackward0>)

确认一下

In [15]:
net[0].weight.data.device

device(type='cuda', index=0)

### 5.6.4 小结

- 我们可以指定用于存储和计算的设备，例如CPU或GPU。默认情况下，数据在主内存中创建，然后使用CPU进行计算。
- 深度学习框架要求计算的所有输入数据都在同一设备上，无论是CPU还是GPU。
- 不经意地移动数据可能会显著降低性能。一个典型的错误如下：计算GPU上每个小批量的损失，并在命令行中将其报告给用户（或将其记录在NumPy ndarray中）时，将触发全局解释器锁，从而使所有GPU阻塞。最好是为GPU内部的日志分配内存，并且只移动较大的日志。